In [1]:
# Imports and Setup
import pandas as pd
import numpy as np
from pathlib import Path
import vectorbt as vbt
from scipy import stats
from scipy.signal import butter, filtfilt
import io
from contextlib import redirect_stdout
from typing import List, Dict, Union
from abc import ABC, abstractmethod
import warnings

# Filter warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)  
warnings.filterwarnings('ignore', category=FutureWarning)   
warnings.filterwarnings('ignore', category=UserWarning)     
np.seterr(all='ignore')  
pd.options.mode.chained_assignment = None  

In [2]:
#Base Strategy Class with Fixed _calculate_zscore
class Strategy:
    """Base class for all trading strategies"""
    
    def __init__(self, df: pd.DataFrame, target_col: str = 'tsx'):
        self.df = df
        self.target_col = target_col
        
    def _calculate_zscore(self, series: pd.Series, window: int = 252) -> pd.Series:
        """Calculate rolling z-score with default window of 252 days"""
        mean = series.rolling(window=window).mean()
        std = series.rolling(window=window).std()
        return (series - mean) / std
        
    def generate_signals(self) -> pd.Series:
        """Generate trading signals (must be implemented by child class)"""
        raise NotImplementedError("Subclass must implement generate_signals()")

In [3]:
#  Buy and Hold Strategy
class BuyAndHoldStrategy(Strategy):
    """Strategy 0: Buy and Hold Strategy
    
    This is the baseline strategy that simply buys and holds the target asset.
    Used as a benchmark for comparing other strategies.
    """
    
    def generate_signals(self) -> pd.Series:
        """Generate constant True signals for buy and hold"""
        return pd.Series(True, index=self.df.index)

In [4]:
# Dual Momentum Strategy:
class DualMomentumStrategy(Strategy):
    """
    Strategy: Dual Momentum Strategy
    
    Logic:
    - Combines absolute and relative momentum
    - Uses multiple lookback periods for robustness
    - Incorporates volatility regime filtering
    """
    
    def __init__(self, df: pd.DataFrame, 
                 lookback_periods: list = [20, 60, 120],
                 vol_window: int = 20,
                 vol_threshold: float = 1.5):
        super().__init__(df)
        self.lookback_periods = lookback_periods
        self.vol_window = vol_window
        self.vol_threshold = vol_threshold
        
    def generate_signals(self) -> pd.Series:
        signals = pd.Series(False, index=self.df.index)
        
        # Calculate returns for each lookback period
        momentum_signals = []
        for period in self.lookback_periods:
            # Absolute momentum (trend following)
            abs_momentum = self.df[self.target_col].pct_change(period) > 0
            
            # Relative momentum (compared to other assets)
            other_assets = [col for col in self.df.columns if col != self.target_col 
                          and 'er_ytd_index' in col]
            
            rel_returns = []
            for asset in other_assets:
                rel_momentum = (self.df[self.target_col].pct_change(period) >
                              self.df[asset].pct_change(period))
                rel_returns.append(rel_momentum)
            
            rel_momentum_signal = pd.concat(rel_returns, axis=1).all(axis=1)
            
            # Combine absolute and relative momentum
            momentum_signals.append(abs_momentum & rel_momentum_signal)
        
        # Combine signals from different lookback periods
        combined_momentum = pd.concat(momentum_signals, axis=1).mean(axis=1) > 0.5
        
        # Volatility regime filter
        vol = self.df[self.target_col].pct_change().rolling(self.vol_window).std() * np.sqrt(252)
        vol_filter = vol <= (vol.rolling(252).mean() * self.vol_threshold)
        
        # Final signal
        signals = combined_momentum & vol_filter
        
        return signals

In [5]:
#Mean Reversion Strategy
class MeanReversionStrategy(Strategy):
    """
    Strategy 3: Mean Reversion Strategy
    
    Logic:
    - Uses Bollinger Bands for mean reversion signals
    - Incorporates RSI for confirmation
    - Filters trades based on volatility regime
    """
    
    def __init__(self, df: pd.DataFrame,
                 bb_window: int = 20,
                 bb_std: float = 2.0,
                 rsi_window: int = 14,
                 rsi_threshold: float = 30):
        super().__init__(df)
        self.bb_window = bb_window
        self.bb_std = bb_std
        self.rsi_window = rsi_window
        self.rsi_threshold = rsi_threshold
        
    def _calculate_rsi(self) -> pd.Series:
        """Calculate RSI indicator"""
        delta = self.df[self.target_col].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=self.rsi_window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=self.rsi_window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
        
    def generate_signals(self) -> pd.Series:
        signals = pd.Series(False, index=self.df.index)
        
        # Calculate Bollinger Bands
        ma = self.df[self.target_col].rolling(window=self.bb_window).mean()
        std = self.df[self.target_col].rolling(window=self.bb_window).std()
        lower_band = ma - (self.bb_std * std)
        
        # Calculate RSI
        rsi = self._calculate_rsi()
        
        # Generate signals
        # Buy when price below lower band and RSI oversold
        signals = (self.df[self.target_col] < lower_band) & (rsi < self.rsi_threshold)
        
        # Add volatility filter
        vol = self.df[self.target_col].pct_change().rolling(window=self.bb_window).std() * np.sqrt(252)
        vol_filter = vol <= vol.rolling(252).mean() * 1.5
        
        return signals & vol_filter

In [6]:
#Multi-Factor Strategy
class MultiFactorStrategy(Strategy):
    """
    Strategy: Multi-Factor Strategy
    
    Logic:
    - Combines value, momentum, volatility, and macro factors
    - Uses dynamic factor weights based on regime
    - Incorporates cross-sectional ranking
    """
    
    def __init__(self, df: pd.DataFrame,
                 lookback: int = 252,
                 n_factors: int = 4):
        super().__init__(df)
        self.lookback = lookback
        self.n_factors = n_factors
        
    def _calculate_value_score(self) -> pd.Series:
        """Calculate value factor score"""
        if 'pe_ratio' in self.df.columns:
            return -self._calculate_zscore(self.df['pe_ratio'])
        return pd.Series(0, index=self.df.index)
    
    def _calculate_momentum_score(self) -> pd.Series:
        """Calculate momentum factor score"""
        return self._calculate_zscore(self.df[self.target_col].pct_change(self.lookback))
    
    def _calculate_volatility_score(self) -> pd.Series:
        """Calculate volatility factor score"""
        vol = self.df[self.target_col].pct_change().rolling(63).std() * np.sqrt(252)
        return -self._calculate_zscore(vol)
    
    def _calculate_macro_score(self) -> pd.Series:
        """Calculate macro factor score"""
        if 'us_economic_regime' in self.df.columns:
            return self._calculate_zscore(self.df['us_economic_regime'])
        return pd.Series(0, index=self.df.index)
    
    def generate_signals(self) -> pd.Series:
        # Calculate factor scores
        value_score = self._calculate_value_score()
        momentum_score = self._calculate_momentum_score()
        vol_score = self._calculate_volatility_score()
        macro_score = self._calculate_macro_score()
        
        # Dynamic factor weights based on regime
        is_risk_on = (self.df['us_economic_regime'] > 0.7) & \
                     (self.df['vix'] < self.df['vix'].rolling(252).mean())
        
        # Adjust weights based on regime
        weights = pd.DataFrame({
            'value': np.where(is_risk_on, 0.1, 0.4),
            'momentum': np.where(is_risk_on, 0.4, 0.1),
            'volatility': np.where(is_risk_on, 0.3, 0.3),
            'macro': np.where(is_risk_on, 0.2, 0.2)
        }, index=self.df.index)
        
        # Combine scores
        combined_score = (weights['value'] * value_score +
                        weights['momentum'] * momentum_score +
                        weights['volatility'] * vol_score +
                        weights['macro'] * macro_score)
        
        # Generate signals based on combined score
        signals = combined_score > combined_score.rolling(self.lookback).mean()
        
        return signals

In [7]:
#Volatility Regime Strategy
class VolatilityRegimeStrategy(Strategy):
    """
    Strategy 5: Volatility Regime & Cross-Asset Strategy
    
    Logic:
    1. Identifies volatility regimes using multiple methods:
       - Realized volatility
       - VIX regime
       - Cross-asset volatility (rates, credit, equity)
    2. Uses volatility surface (term structure) information
    3. Incorporates cross-asset correlations
    4. Adapts position sizing based on risk environment
    """
    
    def __init__(self, df: pd.DataFrame,
                 vol_window: int = 30,
                 correlation_window: int = 90,
                 regime_window: int = 252,
                 vol_threshold: float = 1.2):
        super().__init__(df)
        self.vol_window = vol_window
        self.correlation_window = correlation_window
        self.regime_window = regime_window
        self.vol_threshold = vol_threshold
        
    def _calculate_vol_surface_score(self) -> pd.Series:
        """Calculate volatility surface score using VIX"""
        implied_vol = self.df['vix']
        realized_vols = pd.DataFrame(index=self.df.index)
        
        realized_vols[f'vol_{self.vol_window}'] = self.df[self.target_col] \
            .pct_change().rolling(self.vol_window).std() * np.sqrt(252)
        
        vol_premium = implied_vol - realized_vols.mean(axis=1)
        vol_premium_zscore = (vol_premium - vol_premium.rolling(252).mean()) / \
                            vol_premium.rolling(252).std()
                            
        return -vol_premium_zscore
        
    def _calculate_correlation_score(self) -> pd.Series:
        """Calculate dynamic correlations with other assets"""
        target_returns = self.df[self.target_col].pct_change()
        assets = ['cad_oas', 'us_hy_oas', 'us_ig_oas']
        asset_returns = self.df[assets].pct_change()
        
        correlations = pd.DataFrame(index=self.df.index)
        for asset in assets:
            correlations[asset] = target_returns.rolling(self.correlation_window) \
                .corr(asset_returns[asset])
        
        avg_correlation = correlations.mean(axis=1)
        correlation_zscore = (avg_correlation - avg_correlation.rolling(252).mean()) / \
                           avg_correlation.rolling(252).std()
                           
        return -correlation_zscore
        
    def _calculate_vol_regime(self) -> pd.Series:
        """Identify volatility regime using multiple indicators"""
        assets = ['cad_oas', 'us_hy_oas', 'us_ig_oas']
        vol_indicators = pd.DataFrame(index=self.df.index)
        
        for asset in assets:
            vol = self.df[asset].pct_change().rolling(20).std() * np.sqrt(252)
            vol_indicators[f'{asset}_vol'] = (vol < vol.rolling(252).mean())
            
        vol_indicators['vix_regime'] = self.df['vix'] < self.df['vix'].rolling(252).mean()
        low_vol_regime = vol_indicators.mean(axis=1) > 0.5
        return low_vol_regime
        
    def generate_signals(self) -> pd.Series:
        """Generate trading signals based on volatility regime"""
        vol_surface_score = self._calculate_vol_surface_score()
        correlation_score = self._calculate_correlation_score()
        vol_regime = self._calculate_vol_regime()
        
        returns = self.df[self.target_col].pct_change()
        trend = returns.rolling(60).mean() / returns.rolling(60).std()
        trend_strength = trend.abs()
        
        signals = (
            vol_regime &  # Low volatility regime
            (vol_surface_score > 0) &  # Positive vol surface score
            (correlation_score > -0.3) &  # Not too high correlations
            (trend_strength > 0.1)  # Some trend presence
        )
        
        signals = signals.rolling(5).mean() > 0.6
        return signals

In [8]:
#Adaptive Trend Strategy
class AdaptiveTrendStrategy(Strategy):
    """
    Strategy: Adaptive Trend Strategy
    
    Logic:
    - Uses adaptive trend filters
    - Incorporates market regime detection
    - Adjusts signal generation based on trend strength
    """
    
    def __init__(self, df: pd.DataFrame,
                 short_window: int = 21,
                 long_window: int = 252,
                 trend_threshold: float = 0.05):
        super().__init__(df)
        self.short_window = short_window
        self.long_window = long_window
        self.trend_threshold = trend_threshold
    
    def _calculate_trend_strength(self) -> pd.Series:
        """Calculate trend strength indicator"""
        # Calculate short and long-term trends
        short_ma = self.df[self.target_col].rolling(self.short_window).mean()
        long_ma = self.df[self.target_col].rolling(self.long_window).mean()
        
        # Calculate trend strength
        trend_strength = (short_ma - long_ma) / long_ma
        
        return self._calculate_zscore(trend_strength)
    
    def _calculate_regime_score(self) -> pd.Series:
        """Calculate regime score based on multiple indicators"""
        # Volatility regime
        vol = self.df[self.target_col].pct_change().rolling(self.short_window).std() * np.sqrt(252)
        vol_regime = vol <= vol.rolling(self.long_window).mean()
        
        # Momentum regime
        momentum = self.df[self.target_col].pct_change(self.short_window)
        mom_regime = momentum > 0
        
        # Combine regimes
        regime_score = pd.concat([vol_regime, mom_regime], axis=1).mean(axis=1)
        
        return regime_score
    
    def generate_signals(self) -> pd.Series:
        # Calculate trend strength
        trend_strength = self._calculate_trend_strength()
        
        # Calculate regime score
        regime_score = self._calculate_regime_score()
        
        # Generate signals
        signals = (trend_strength > self.trend_threshold) & (regime_score > 0.5)
        
        return signals

In [9]:
#Combined Strategy
class CombinedStrategy(Strategy):
    """
    Strategy: Combined Strategy
    
    Logic:
    - Combines signals from multiple strategies
    - Uses dynamic weights based on market conditions
    - Incorporates regime-based risk management
    """
    
    def __init__(self, df: pd.DataFrame):
        super().__init__(df)
        
        # Initialize component strategies
        self.trend_strategy = AdaptiveTrendStrategy(df)
        self.vol_strategy = VolatilityRegimeStrategy(df)
        self.factor_strategy = MultiFactorStrategy(df)
    
    def _calculate_strategy_weights(self) -> dict:
        """Calculate dynamic strategy weights based on market conditions"""
        vix = self.df['vix']
        vol_regime = vix <= vix.rolling(252).mean()
        
        if vol_regime.iloc[-1]:
            # Low volatility regime: favor trend and factor
            weights = {
                'trend': 0.4,
                'vol': 0.2,
                'factor': 0.4
            }
        else:
            # High volatility regime: favor volatility strategy
            weights = {
                'trend': 0.2,
                'vol': 0.5,
                'factor': 0.3
            }
            
        return weights
    
    def generate_signals(self) -> pd.Series:
        # Generate individual strategy signals
        trend_signal = self.trend_strategy.generate_signals()
        vol_signal = self.vol_strategy.generate_signals()
        factor_signal = self.factor_strategy.generate_signals()
        
        # Calculate strategy weights
        weights = self._calculate_strategy_weights()
        
        # Combine signals using weights
        combined = (
            weights['trend'] * trend_signal.astype(float) +
            weights['vol'] * vol_signal.astype(float) +
            weights['factor'] * factor_signal.astype(float)
        )
        
        return combined > 0.5

In [10]:
#Macro Regime Strategy
class MacroRegimeStrategy(Strategy):
    """Strategy 2: Macro Regime-Based Strategy"""
    
    def __init__(self, df: pd.DataFrame, 
                 lookback: int = 252,
                 z_threshold: float = 1.0):
        super().__init__(df)
        self.lookback = lookback
        self.z_threshold = z_threshold
        
    def generate_signals(self) -> pd.Series:
        signals = pd.Series(False, index=self.df.index)
        
        # 1. VIX regime
        vix_zscore = self._calculate_zscore(self.df['vix'], window=self.lookback)  # Added window parameter
        vix_regime = vix_zscore < self.z_threshold
        
        # 2. Yield curve regime
        curve_zscore = self._calculate_zscore(self.df['us_3m_10y'], window=self.lookback)  # Added window parameter
        curve_regime = curve_zscore > -self.z_threshold
        
        # 3. Economic surprises regime
        surprise_indicators = ['us_growth_surprises', 'us_inflation_surprises', 
                             'us_hard_data_surprises']
        surprise_regimes = []
        
        for indicator in surprise_indicators:
            if indicator in self.df.columns:
                zscore = self._calculate_zscore(self.df[indicator], window=self.lookback)  # Added window parameter
                surprise_regimes.append(zscore > 0)
        
        if surprise_regimes:
            combined_surprise_regime = pd.concat(surprise_regimes, axis=1).mean(axis=1) > 0.5
        else:
            combined_surprise_regime = pd.Series(True, index=self.df.index)
        
        # 4. Economic regime check
        if 'us_economic_regime' in self.df.columns:
            econ_regime = self.df['us_economic_regime'] > 0
        else:
            econ_regime = pd.Series(True, index=self.df.index)
        
        # Combine all regimes
        signals = vix_regime & curve_regime & combined_surprise_regime & econ_regime
        
        return signals

In [11]:
# Backtesting Framework
class BacktestConfig:
    def __init__(self, 
                 start_date=None, 
                 end_date=None,
                 rebalance_freq='1D',  # '1D' for daily, 'M' for monthly
                 initial_capital=100,
                 size=1.0,
                 size_type='percent'):
        self.start_date = pd.to_datetime(start_date) if start_date else None
        self.end_date = pd.to_datetime(end_date) if end_date else None
        self.rebalance_freq = rebalance_freq
        self.initial_capital = initial_capital
        self.size = size
        self.size_type = size_type

def load_data(config: BacktestConfig) -> pd.DataFrame:
    data_path = Path('..') / 'raw_data' / 'df.csv'
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    if config.start_date:
        df = df[df.index >= config.start_date]
    if config.end_date:
        df = df[df.index <= config.end_date]
        
    return df

def create_portfolio(strategy, price, signals, config: BacktestConfig):
    signals = signals.astype(bool)
    
    if config.rebalance_freq != '1D':
        monthly_signals = signals.resample('M').last()
        signals = monthly_signals.reindex(price.index, method='ffill')
        signals = signals.astype(bool)
    
    entries = signals & ~signals.shift(1).fillna(False)
    exits = ~signals & signals.shift(1).fillna(False)
    
    return vbt.Portfolio.from_signals(
        price,
        entries,
        exits,
        freq='1D',
        init_cash=config.initial_capital,
        size=config.size,
        size_type=config.size_type,
        accumulate=False
    )

def format_results(stats_dict):
    """
    Format and style the backtest results for display
    """
    # Convert to DataFrame and sort by Total Return
    df_stats = pd.DataFrame.from_dict(stats_dict, orient='index').T
    df_stats = df_stats.sort_values(by='Total Return [%]', axis=1, ascending=False)
    
    # Reorder metrics to a more logical sequence
    ordered_metrics = [
        'Start',
        'End',
        'Period',
        'Start Value',
        'End Value',
        'Total Return [%]',
        'Annualized Return [%]',
        'Annualized Volatility [%]',
        'Sharpe Ratio',
        'Sortino Ratio',
        'Calmar Ratio',
        'Max Drawdown [%]',
        'Max Drawdown Duration',
        'Win Rate [%]',
        'Best Trade [%]',
        'Worst Trade [%]',
        'Avg Winning Trade [%]',
        'Avg Losing Trade [%]',
        'Avg Winning Trade Duration',
        'Avg Losing Trade Duration',
        'Profit Factor',
        'Expectancy',
        'Total Trades',
        'Total Closed Trades',
        'Total Open Trades',
        'Open Trade PnL',
    ]
    
    # Only keep metrics that exist in the DataFrame
    ordered_metrics = [m for m in ordered_metrics if m in df_stats.index]
    df_stats = df_stats.reindex(ordered_metrics)
    
    # Format specific columns
    formatted_df = df_stats.copy()
    
    # Format dates
    date_rows = ['Start', 'End']
    for row in date_rows:
        if row in formatted_df.index:
            formatted_df.loc[row] = formatted_df.loc[row].apply(
                lambda x: pd.to_datetime(x).strftime('%m/%d/%Y') if pd.notnull(x) else x
            )
    
    # Format percentage returns
    percentage_rows = [
        'Total Return [%]', 'Annualized Return [%]', 'Annualized Volatility [%]',
        'Max Drawdown [%]', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
        'Avg Winning Trade [%]', 'Avg Losing Trade [%]'
    ]
    percentage_rows = [row for row in percentage_rows if row in formatted_df.index]
    for row in percentage_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{x:.2f}%" if pd.notnull(x) else x
        )
    
    # Format monetary values
    monetary_rows = ['Start Value', 'End Value', 'Open Trade PnL']
    monetary_rows = [row for row in monetary_rows if row in formatted_df.index]
    for row in monetary_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{x:,.2f}" if pd.notnull(x) else x
        )
    
    # Format ratios to 2 decimal places
    ratio_rows = ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Profit Factor', 'Expectancy']
    ratio_rows = [row for row in ratio_rows if row in formatted_df.index]
    for row in ratio_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{x:.2f}" if pd.notnull(x) else x
        )
    
    # Format durations
    duration_rows = [
        'Period', 'Max Drawdown Duration', 
        'Avg Winning Trade Duration', 'Avg Losing Trade Duration'
    ]
    duration_rows = [row for row in duration_rows if row in formatted_df.index]
    for row in duration_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{pd.Timedelta(x).days} days" if pd.notnull(x) and isinstance(x, (pd.Timedelta, str)) else x
        )
    
    # Format integer values
    integer_rows = ['Total Trades', 'Total Closed Trades', 'Total Open Trades']
    integer_rows = [row for row in integer_rows if row in formatted_df.index]
    for row in integer_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{int(x):,}" if pd.notnull(x) else x
        )
    
    # Apply styling
    styled_df = formatted_df.style.set_properties(**{
        'text-align': 'center',
        'padding': '5px',
        'border': '1px solid #ddd'
    }).set_table_styles([
        {'selector': 'th', 'props': [
            ('text-align', 'center'),
            ('background-color', '#f5f5f5'),
            ('font-weight', 'bold'),
            ('padding', '5px'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': '', 'props': [
            ('border-collapse', 'collapse'),
            ('border', '1px solid #ddd')
        ]}
    ])
    
    # Add hover effect
    styled_df = styled_df.set_table_styles([
        {'selector': 'tr:hover', 'props': [('background-color', '#f0f0f0')]},
    ], overwrite=False)
    
    # Highlight best values in each numeric row
    numeric_rows = percentage_rows + ratio_rows + monetary_rows
    for row in numeric_rows:
        if row in formatted_df.index:
            row_values = pd.to_numeric(formatted_df.loc[row].str.rstrip('%').str.replace(',', ''), errors='coerce')
            if row in ['Max Drawdown [%]', 'Worst Trade [%]']:
                # For these metrics, lower is better
                best_value = row_values.min()
                mask = row_values == best_value
            else:
                # For other metrics, higher is better
                best_value = row_values.max()
                mask = row_values == best_value
            
            styled_df = styled_df.apply(lambda _: ['background-color: #e6ffe6' if v else '' for v in mask], axis=1, subset=[row])
    
    return styled_df

def run_backtest(strategies, config: BacktestConfig):
    """
    Run backtest for multiple strategies and format results
    """
    # Dictionary to store stats for each strategy
    all_stats = {}
    
    # Load and filter data once according to config
    df = load_data(config)
    
    # Create new instances of strategies with filtered data
    strategy_instances = [
        strategy.__class__(df) for strategy in strategies
    ]
    
    for strategy in strategy_instances:
        with redirect_stdout(io.StringIO()):
            signals = strategy.generate_signals()
        
        price = strategy.df[strategy.target_col]
        signals = signals.reindex(price.index)
        
        # Ensure signals are boolean
        if not isinstance(signals.dtype, pd.BooleanDtype):
            signals = signals.astype(bool)
        
        # Create portfolio using config
        pf = create_portfolio(strategy, price, signals, config)
        
        # Get portfolio stats
        stats_series = pf.stats()
        
        # Get returns stats
        returns = pf.returns()
        returns_stats = returns.vbt.returns(freq='1D', year_freq='365D')
        
        # Add annualized metrics
        stats_series['Annualized Return [%]'] = returns_stats.annualized() * 100
        stats_series['Annualized Volatility [%]'] = returns_stats.annualized_volatility() * 100
        
        # Calculate additional metrics
        stats_series['Sharpe Ratio'] = returns_stats.sharpe_ratio()
        stats_series['Sortino Ratio'] = returns_stats.sortino_ratio()
        stats_series['Calmar Ratio'] = returns_stats.calmar_ratio()
        
        all_stats[strategy.__class__.__name__] = stats_series
    
    return format_results(all_stats)



In [13]:
# Run Backtest
# Initialize strategies
df = load_data(BacktestConfig())
strategies = [
    BuyAndHoldStrategy(df),
    DualMomentumStrategy(df),
    MacroRegimeStrategy(df),
    MeanReversionStrategy(df),
    MultiFactorStrategy(df),
    VolatilityRegimeStrategy(df),
    AdaptiveTrendStrategy(df),
    CombinedStrategy(df)
]

# Test different periods
config_default = BacktestConfig()
results_default = run_backtest(strategies, config_default)

# Test 2020 period with monthly rebalancing
config_2020 = BacktestConfig(
    start_date='2000-01-01',
    end_date='2025-12-31',
    rebalance_freq='M'
)
results_2020 = run_backtest(strategies, config_2020)

# Display results
# Convert Styler to DataFrame first if needed
if hasattr(results_default, 'data'):
    results_default = results_default.data
if hasattr(results_2020, 'data'):
    results_2020 = results_2020.data

# Now display the results
display(results_default)
display(results_2020)

,BuyAndHoldStrategy,MultiFactorStrategy,DualMomentumStrategy,AdaptiveTrendStrategy,MacroRegimeStrategy,MeanReversionStrategy,VolatilityRegimeStrategy,CombinedStrategy
Start,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002
End,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024
Period,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days
Start Value,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
End Value,396.82,224.55,222.40,155.41,141.01,96.54,90.69,87.94
Total Return [%],296.82%,124.55%,122.40%,55.41%,41.01%,-3.46%,-9.31%,-12.06%
Annualized Return [%],9.47%,5.45%,5.39%,2.94%,2.28%,-0.23%,-0.64%,-0.84%
Annualized Volatility [%],19.93%,11.22%,8.35%,6.92%,8.48%,3.03%,3.31%,2.57%
Sharpe Ratio,0.55,0.53,0.67,0.45,0.31,-0.06,-0.18,-0.31
Sortino Ratio,0.75,0.74,0.92,0.63,0.42,-0.08,-0.23,-0.40


,BuyAndHoldStrategy,MultiFactorStrategy,DualMomentumStrategy,AdaptiveTrendStrategy,MeanReversionStrategy,MacroRegimeStrategy,VolatilityRegimeStrategy,CombinedStrategy
Start,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002
End,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024
Period,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days
Start Value,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
End Value,396.82,202.97,182.49,153.07,120.21,115.99,112.09,107.44
Total Return [%],296.82%,102.97%,82.49%,53.07%,20.21%,15.99%,12.09%,7.44%
Annualized Return [%],9.47%,4.76%,4.03%,2.83%,1.22%,0.98%,0.75%,0.47%
Annualized Volatility [%],19.93%,11.53%,9.51%,7.23%,2.29%,8.79%,2.96%,2.24%
Sharpe Ratio,0.55,0.46,0.46,0.42,0.54,0.15,0.27,0.22
Sortino Ratio,0.75,0.64,0.63,0.58,0.87,0.21,0.38,0.32
